<h1>Linjär reggression</h1>
<h2>Analys av california housing dataset</h2>  
<em>Adrian Söderberg Skog</em><br>
<strong>AIMG25</strong>
<h3>Importer</h3>
<p></p>

In [260]:
import numpy as np
import pandas as pd
import LinearRegression as lreg

<h3>Ladda in och inspektera dataset</h3>

In [ ]:
df = pd.read_csv("housing.csv")
df.head()

In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [263]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

<h3>Bearbetning av data</h3>
<p>Datasetet innehåller 207 NaN värden i kolumnen "total bedrooms". Eftersom det är en liten del av de totala antalet rader på 20640, gör jag bedömningen att det är relativt säkert att ta bort dessa. </p>

In [264]:
df_clean = df.dropna()



<p>Vi kan också se att det finns en kolumn med kategoriska värden. För att kunna inkludera dessa i reggressionen behöver de kodas om till numeriska värden. Eftersom de inte har någon tydlig hirarkisk struktur gör jag bedömningen att det lämpligaste är att använda one hot encoding. I min kod fil har jag därför också gjort en klass oneHotEncoding som kan omvandla den kategoriska kolumnen till nya kolumner med binära värden. </p>

In [265]:
encoder = lreg.OneHotEncoder()

y = df_clean.iloc[:,8].to_numpy()
X = pd.concat([df_clean.iloc[:,:8], df_clean.iloc[:,-1]], axis=1) 

X_enc = encoder.fit_transform(X, column_names=df_clean.columns)

<p>Sista steget innan vi kan göra en linjär regressionsanalys på datasetet är att det behöver delas upp i träningsdata och testdata samt att målvariabeln behöver separeras från de övriga parametrarna. I kodfilen har jag gort en funktion som slumpmässigt delar upp punkterna. Defaultuppdelningen är 80% i träningsdata och 20% i testdata. </p>
<p>Målvariabeln kommer att vara median house value, eftersom det är huspriser vi vill kunna förutsäga med hjälp av de andra parametrarna. </p>

In [266]:
data = np.column_stack((X_enc, y))

train_Data, test_Data = lreg.train_test_split(data)

train_X, train_y = train_Data[:,:-1], train_Data[:,-1]
test_X, test_y = test_Data[:,:-1], test_Data[:,-1]

feature_names = encoder.feature_names

<h3>Träning av modellen</h3>


<p>Fit metoden i LinearRegression klassen upskattar koefficienterna med hjälp av minsta kvadratmetoden. Istället för den vanliga formeln <br> &beta; = (X<sup>T</sup>X)<sup>-1</sup>X<sup>T</sup>Y <br> valde jag att använda Moore Penrose pseudoinvers som kan skrivas som X<sup>+</sup> = (X<sup>T</sup>X)<sup>-1</sup>X<sup>T</sup> vilket ger att &beta; = X<sup>+</sup>Y <br>
Fördelen med den är att den fungerar även i situationer där inversen av X<sup>T</sup>X inte är deffinerad, vilket kan uppstå till exempel om det finns multikoliniäritet mellan parametrar eller om X har fler parametrar än rader. <br>
Fit metoden returnerar sedan koefficientmatrisen och interceptet. </p>


In [267]:
model = lreg.LinearRegression()
model.fit(train_X, train_y)

(array([-2.67480638e+04, -2.51468759e+04,  1.04986199e+03, -5.95086544e+00,
         9.72030275e+01, -3.58682606e+01,  4.65767759e+01,  3.89443093e+04,
        -4.04862306e+04,  1.70371986e+05, -3.77571860e+03,  3.55438951e+03]),
 np.float64(-2272629.668018824))

In [268]:
model.summary(feature_names=feature_names, 
              target_name="median house value", 
              testX= test_X, 
              testy= test_y, 
              CL=0.95)

Linear regression summary
Target variable:            median house value
Sample size:                16346
Number of features:         12

variance:                   4729466958.6872
R square:                   0.6429
Adjusted R square:          0.6426
train RMSE:                 68771.1201
test RMSE:                  68274.0642
F-statistic:                2450.5325
p value (F-statistic):      0.0000

Confidence level:   95.0%
Feature                 CI lower   coefficient   CI upper  t-value   p-value   significant
longitude                  -28987.88 -26748.06 -24508.25    -23.41      0.00       Yes
latitude                   -27351.65 -25146.88 -22942.11    -22.36      0.00       Yes
housing_median_age            953.46   1049.86   1146.26     21.35      0.00       Yes
total_rooms                    -7.68     -5.95     -4.22     -6.74      0.00       Yes
total_bedrooms                 82.09     97.20    112.32     12.61      0.00       Yes
population                    -38.18    -35

In [269]:
names = feature_names + ["median house value"]
model.corr_matrix(data, column_names=names)


Pearson correlation:

longitude                  [' 1.00', '-0.92', '-0.11', ' 0.05', ' 0.07', ' 0.10', ' 0.06', '-0.02', '-0.06', ' 0.01', '-0.47', ' 0.05', '-0.05']
latitude                   ['-0.92', ' 1.00', ' 0.01', '-0.04', '-0.07', '-0.11', '-0.07', '-0.08', ' 0.35', '-0.02', ' 0.36', '-0.16', '-0.14']
housing_median_age         ['-0.11', ' 0.01', ' 1.00', '-0.36', '-0.32', '-0.30', '-0.30', '-0.12', '-0.24', ' 0.02', ' 0.26', ' 0.02', ' 0.11']
total_rooms                [' 0.05', '-0.04', '-0.36', ' 1.00', ' 0.93', ' 0.86', ' 0.92', ' 0.20', ' 0.03', '-0.01', '-0.02', '-0.01', ' 0.13']
total_bedrooms             [' 0.07', '-0.07', '-0.32', ' 0.93', ' 1.00', ' 0.88', ' 0.98', '-0.01', '-0.01', '-0.00', '-0.02', ' 0.00', ' 0.05']
population                 [' 0.10', '-0.11', '-0.30', ' 0.86', ' 0.88', ' 1.00', ' 0.91', ' 0.01', '-0.02', '-0.01', '-0.06', '-0.02', '-0.03']
households                 [' 0.06', '-0.07', '-0.30', ' 0.92', ' 0.98', ' 0.91', ' 1.00', ' 0.01', '-0.04',

<h3>Resultat och disskusion</h3>
<p>
Överlag kan man konstatera att reggressionen är signifikant. Vi har en förklaringsgrad på nära 65% dvs. modellen kan förklara 65% av variationen i huspriserna, vilket är ganska okej med tanke på att det finns många fler faktorer som kan påverka huspriser. R<sup>2</sup> och adjusted R<sup>2</sup> är i princip identistka vilket tyder på att vi inte har övertränat med onödiga parametrar. Tränings RMSE och test RMSE är också relativt lika vilket också indikerar att modellen generaliserar bra. Modellen missar i snitt med 68000$. <br>
Jag har valt en konfidensnivå på 95%. Det finns ingen ingen anledning att vara striktare än så vid estimering av fastighetspriser. Detta innebär att vi har en signifikansnivå på 5% 
F testet gav ett p värde som är väldigt nära 0, långt under 0,05 vilket gör att vi med säkerhet kan avfärda nollhypetesen. Med andra ord så vet vi att minst en av parametrarna har en signifikant effekt. Kollar vi på de individuella parametrarna så ser vi också att det bara är NEAR BAY och NEAR OCEAN som är i riskzoonen för att inte vara signiffikanta där NEAR OCEAN hamnade precis under 0,05.<br> Vår bästa predikator är median income som har högst signiffikans. Vi kan också se i korrelationsmatrisen att den har en korrelation med median house value på 0.69. Det är inte helt oväntat eftersom folk med höre inkomst i större utsträckning kan köpa dyrare hus. Det är också intressant att antalet totala rum har en negativ effekt på priset men totala antalet sovrum har en positiv effekt. Men eftersom det är totala antalet rum i ett område så kan man tänka sig att ett stort antal rum också kan innebära många små hus som tenderar att vara billigare. <br>
</p>
<p>
I korrelationsmatrisen kan vi se tendenser till multikoliniäritet mellan parametrarna total rooms, total bedrooms, population och households. I detta läge kanske man skulle valt ut en av dessa som är mest relevant, men användandet av Moore Penrose pseudoinvers skall hjälpa till att få stabilare uträkning av koefficienter i dessa lägen. Hade vi använt den vanliga formeln så hade vi förmodligen sett större varians i kofficienterna. 
</p>
<p>
Sammanfattningsvis kan vi konstatera att modellen kan ge relevanta prediktioner. Träningsdatan består av 16346 rader, vilket är tillräkligt för att kunna göra en bra generalisering, och det såg vi också i resultatet. För att lättare kunna gämföra parametrarnas signifikans hade det varit födelaktigt att normalisera datan innan. Det tar jag med mig till nästa gång.    
</p>